# Section 1: Data Imports and Manipulation

This section covers:

* Initial data imports for main variables and controls
* Data cleaning
* Manipulation


### 1. Monetary Factors

Variables that capture policy stance and liquidity environment:

* ON RRP award rate: The rate paid by the Fed on overnight reverse repos; measures the attractiveness of the facility.
* EFFR
* 3-month Treasury bill rate: Proxy for short-term market yields and the opportunity cost of holding cash or using the ON RRP.
* Reserves at the Fed: Indicates total liquidity available in the banking system.
* QT dummy: Equals 1 during Quantitative Tightening (March 2022–present), 0 otherwise. QT periods are when the Fed reduces its balance sheet, shrinking reserves and increasing liquidity competition among banks. Including this helps capture how the ON RRP’s effects may change under reserve scarcity compared to QE or neutral periods.

(The IORB rate is excluded due to high correlation with EFFR, which already reflects policy stance.)

### 2. Bank and Macro Factors

Macro-level variables to control for business cycle conditions and market stress:

* Inflation (CPI): Controls for price-level effects and nominal rate adjustments.
* MOVE Index: Measures Treasury market volatility and uncertainty; proxy for financial stress.
* VIX: Captures general market uncertainty and risk sentiment beyond fixed-income markets.
* Capital Ratios: measures how much capital banks hold relative to the riskiness of their assets — higher values mean banks are financially stronger and need to hold less cash as a safety buffer. When the ON RRP rate rises, it can pull reserves out of the banking system. Banks with higher capital ratios can handle that drain more easily—they don’t need to rush to replace lost liquidity. Banks with lower capital ratios may respond by tightening liquidity further or hoarding cash to stay safe.
* Securities to asset ratio: Captures how much of a bank’s balance sheet is invested in marketable securities—essentially a substitute for holding cash. Higher values mean the bank can quickly sell assets for cash if needed, so it keeps fewer idle reserves. Lower values imply fewer liquid assets, so the bank relies more on cash balances.When the ON RRP rate increases or usage rises, money often flows from bank deposits into the facility, shrinking bank reserves. Banks with high securities holdings can offset that by selling securities.Banks with low securities holdings feel more pressure on their liquidity ratios when ON RRP drains reserves.
  

### 3. Key Interaction Terms

Used to test state-dependent effects—whether the ON RRP’s influence varies with financial stress or reserve scarcity:

* ON RRP Rate × MOVE: Tests whether the ON RRP’s rate becomes more attractive to investors and banks during times of Treasury market volatility.
* ON RRP Volume × MOVE: Examines whether ON RRP take-up increases when volatility rises, signaling flight-to-safety behavior.

### 4. Hypothesis

The ON RRP acts as a marginal liquidity facility that stabilizes short-term rates instead of directly altering banks’ balance sheets. Expect a small average effect on banks’ cash-to-asset ratios under normal conditions. Expect stronger effects during periods of high market volatility (MOVE, VIX) or reserve scarcity (QT periods). Consider robustness tests by splitting the sample into “ample reserves” (2015–2019) vs. “scarce reserves” (2022–2024).

In [50]:
!pip install statsmodels

In [191]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.ardl import ardl_select_order


# Section 1: Data Importation and Cleaning

* Import data from FRED and Investing site
* Clean up series names
* Add interaction variables
* Log variables
* VIF Tests
* ADF Tests

In [214]:
# award volumn
award_df = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=RRPONTSYD&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=2003-02-07")

# cash from commercial banks
cash_df = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CASACBW027SBOG&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1973-01-03")

# assets from commercial banks
total_asset_df = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=TLAACBW027SBOG&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1973-01-03")

# award rate from FED ON-RRP
award_rate = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=RRPONTSYAWARD&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=2013-09-23")

# inflation
cpi_inflation = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPILFESL&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=pch&vintage_date=2025-10-07&revision_date=2025-10-07&nd=1957-01-01")

# iorb rate
effr = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=FEDFUNDS&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1954-07-01")

# reserves at fed data to see measure of system liquidity
reserves = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=TOTRESNS&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1959-01-01")

# treasury bill rate 3-months
tbill_rate = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=TB3MS&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1934-01-01")

In [215]:
# import market stress variables
move = pd.read_csv(r"C:\Users\khang\Downloads\ICE BofAML MOVE Historical Data (3).csv")
move['Date'] = move['Date'].str.replace('/', '-')
move['Date'] = pd.to_datetime(move['Date'], format='%m-%d-%Y')
move = move[['Date', 'Price']]
move['Date'] = move['Date'].dt.strftime('%Y-%m-%d')

# Ensure Date column is datetime type and sorted
move['Date'] = pd.to_datetime(move['Date'])
move = move.sort_values('Date')

# Set Date as index for resampling
move = move.set_index('Date')

# Resample to quarterly frequency using the mean of each quarter
move = move.resample('Q')['Price'].mean().reset_index()

# Convert to end-of-quarter timestamps for neat labeling (e.g. 2024-03-31)
move['Date'] = move['Date'].dt.to_period('Q').dt.to_timestamp()
move['Date'] = move['Date'].dt.strftime('%Y-%m-%d')

# import VIX variable
vix = pd.read_csv(r"https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=VIXCLS&scale=left&cosd=2014-01-01&coed=2024-12-31&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-29&revision_date=2025-10-29&nd=1990-01-02")
vix = vix.rename(columns={'VIXCLS': 'vix'})

C:\Users\khang\AppData\Local\Temp\ipykernel_34000\1505101318.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  move = move.resample('Q')['Price'].mean().reset_index()


In [216]:
# import more bank level variables
# capital ratio
capital_ratio_df = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=BOGZ1FL010000016Q&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-10-22&revision_date=2025-10-22&nd=2009-10-01")
capital_ratio_df = capital_ratio_df.rename(columns={'BOGZ1FL010000016Q': 'capital_ratio'})

# quartlerly data lists months for 01, 04, 07, 10. figure out how to have the same ratio values for the preceding months after the quarter is listed. (ex: 02, 03 would be 13.191)
capital_ratio_df["observation_date"] = pd.to_datetime(capital_ratio_df["observation_date"])
capital_ratio_df['observation_date'] = capital_ratio_df['observation_date'].dt.strftime('%Y-%m-%d')

# download equity
equity = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=QBPBSTLKTEQKTBKEQK&scale=left&cosd=2014-01-01&coed=2024-12-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=nbd&vintage_date=2025-10-22&revision_date=2025-10-22&nd=1984-01-01")
equity = equity.rename(columns={'QBPBSTLKTEQKTBKEQK_NBD19840101': 'equity'})
equity = equity.merge(total_asset_df, on='observation_date')

equity['eq_asset_r'] = equity['equity'] / equity['TLAACBW027SBOG']
equity = equity.drop(columns=['TLAACBW027SBOG', 'equity'])

In [217]:
cash_to_asset_df = pd.merge(cash_df, total_asset_df, how='inner', on='observation_date')
cash_to_asset_df = cash_to_asset_df.rename(columns={'TLAACBW027SBOG': 'total_assets', 'CASACBW027SBOG': 'cash_assets'})
cash_to_asset_df['ratio'] = cash_to_asset_df['cash_assets'] / cash_to_asset_df['total_assets']

df = pd.merge(cash_to_asset_df, award_df, how='inner', on='observation_date')
df = df.rename(columns={'RRPONTSYD': 'total_takeup'})
df['ratio'] = df['cash_assets'] / df['total_assets']
df = df.merge(award_rate, on='observation_date')
df = df.rename(columns={'RRPONTSYAWARD': 'award_rate'})

In [218]:
# join iorb df
df = df.merge(effr, on='observation_date')
df = df.rename(columns={'FEDFUNDS': 'effr'})

# clean and join cpi
df = df.merge(cpi_inflation, on='observation_date')
df = df.rename(columns={'CPILFESL_PCH':'cpi'})

# join move
df = df.merge(move, left_on='observation_date', right_on='Date')
df = df.drop(columns=['Date'])
df = df.rename(columns={'Price':'move'})

# join vix
df = df.merge(vix, on='observation_date')

# join reserves and tbill rate
df = df.merge(reserves, on='observation_date')
df = df.rename(columns={'TOTRESNS':'reserves'})

# join tbill rate
df = df.merge(tbill_rate, on='observation_date')
df = df.rename(columns={'TB3MS':'tbill_rate3m'})

# join capital asset ratio
df = df.merge(capital_ratio_df, on='observation_date')

# join equity to asset ratio
df = df.merge(equity, on='observation_date')


In [219]:
df = df.drop(columns=['cash_assets', 'total_assets'])

In [220]:
# log transform move, vix, reserves at fed, total takeup
df['ln_move'] = np.log(df['move'])
df['ln_vix'] = np.log(df['vix'])
df['ln_reserves'] = np.log(df['reserves'])
df['ln_total_takeup'] = np.log(df['total_takeup'])

In [221]:
# interactions
df['ln_award_rate_move'] = df['award_rate'] * df['move']
df['ln_takeup_move'] = df['total_takeup'] * df['move']

In [222]:
# create QT dummy: 1 after March 2022, else 0
df['QT_dummy'] = (df['observation_date'] >= '2022-03-01').astype(int)

## VIF Tests and Rate Spread Variable

In [223]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def compute_vif(df, cols):
    """
    Computes Variance Inflation Factors (VIF) for a set of variables.

    Parameters:
    df   : pandas DataFrame containing the variables
    cols : list of strings of column names for the VIF regression

    Returns:
    DataFrame with VIF values for each variable
    """

    # Build matrix including constant
    X = df[cols].dropna()
    X = sm.add_constant(X)

    vif_data = []
    
    for i in range(X.shape[1]):
        vif = variance_inflation_factor(X.values, i)
        vif_data.append({
            "variable": X.columns[i],
            "VIF": vif
        })

    return pd.DataFrame(vif_data)


In [224]:
cols_to_test = [
    "ratio",
    "ln_total_takeup",
    "capital_ratio",
    "award_rate",
    "tbill_rate3m",
    "eq_asset_r",
    "cpi",
    "ln_vix"
]

vif_results = compute_vif(df, cols_to_test)
print(vif_results)

          variable           VIF
0            const  11750.596074
1            ratio      3.518126
2  ln_total_takeup      3.143603
3    capital_ratio      3.640564
4       award_rate    253.228732
5     tbill_rate3m    277.383426
6       eq_asset_r     11.906766
7              cpi      1.368357
8           ln_vix      3.098076


High multicollinearity between award_rate and tbill_rate3m calls for rate_spread variable creation

In [225]:
#use a rate spread and replace tbill3m to capture relative incentives
df['rate_spread'] = df['award_rate'] - df['tbill_rate3m']

In [226]:
cols_to_test = [
    "ratio",
    "ln_total_takeup",
    "capital_ratio",
    "award_rate",
    "eq_asset_r",
    "cpi",
    "rate_spread",
    "ln_vix"
]

vif_results = compute_vif(df, cols_to_test)
print(vif_results)


          variable           VIF
0            const  11750.596074
1            ratio      3.518126
2  ln_total_takeup      3.143603
3    capital_ratio      3.640564
4       award_rate      5.464522
5       eq_asset_r     11.906766
6              cpi      1.368357
7      rate_spread      1.759432
8           ln_vix      3.098076


## Split data into pre and post 2021 for later analysis

In [227]:
# Ensure your date column is datetime
df['observation_date'] = pd.to_datetime(df['observation_date'])

# Split into pre-2021 and post-2021 samples
df_pre  = df[df['observation_date'] <  '2021-01-01'].copy()
df_post = df[df['observation_date'] >= '2021-01-01'].copy()

# Check sample sizes
print("Pre-2021 observations:", len(df_pre))
print("Post-2021 observations:", len(df_post))


Pre-2021 observations: 28
Post-2021 observations: 16


The equity-to-asset ratio exhibits multicollinearity with other balance-sheet variables, which is expected given shared denominators in bank ratio construction. This does not affect model validity because ARDL–ECM methods are robust to correlated regressors, and all models remain stable and cointegrated.

## ADF Tests

Prove each series is I(0) or I(1) to make sure the use of ARDL is valid

In [228]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

def adf_order_summary(df, cols, alpha=0.05, maxlag=None, regression='c'):
    """
    Run ADF tests on all specified variables in levels and first differences,
    and classify each series as I(0), I(1), or inconclusive.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the time-series variables.
    cols : list of str
        Column names to test.
    alpha : float
        Significance level for stationarity decision (default 0.05).
    maxlag : int or None
        Max lag for ADF (None = AIC-based automatic selection).
    regression : str
        'c' (constant), 'ct' (constant + trend), etc. (ADFuller regression type).

    Returns
    -------
    summary : pandas.DataFrame
        Table with test statistics, p-values, and I(d) classification.
    """

    results = []

    for col in cols:
        series = df[col].dropna()

        # --- ADF on levels ---
        adf_level = adfuller(series, maxlag=maxlag, regression=regression, autolag='AIC')
        stat_level, p_level, usedlag_level, nobs_level, crit_level, icbest_level = adf_level

        # --- ADF on first differences ---
        diff_series = series.diff().dropna()
        adf_diff = adfuller(diff_series, maxlag=maxlag, regression=regression, autolag='AIC')
        stat_diff, p_diff, usedlag_diff, nobs_diff, crit_diff, icbest_diff = adf_diff

        # --- Classification ---
        if p_level < alpha:
            order = "I(0)"
        elif p_level >= alpha and p_diff < alpha:
            order = "I(1)"
        else:
            order = "inconclusive / higher order"

        results.append({
            "variable": col,
            "ADF stat (level)": stat_level,
            "p-value (level)": p_level,
            "lags (level)": usedlag_level,
            "ADF stat (diff)": stat_diff,
            "p-value (diff)": p_diff,
            "lags (diff)": usedlag_diff,
            "order": order
        })

    summary = pd.DataFrame(results)
    return summary


In [229]:
cols_to_test = [
    "ratio",
    "ln_total_takeup",
    "capital_ratio",
    "award_rate",
    "eq_asset_r",
    "cpi",
    "rate_spread",
    "ln_vix"
]

adf_results = adf_order_summary(df, cols_to_test, alpha=0.05)
print(adf_results.round(4))


          variable  ADF stat (level)  p-value (level)  lags (level)  \
0            ratio           -2.5757           0.0981             2   
1  ln_total_takeup           -2.3547           0.1549             1   
2    capital_ratio           -0.5953           0.8720             0   
3       award_rate           -0.5024           0.8916            10   
4       eq_asset_r           -0.7112           0.8438             0   
5              cpi           -1.8361           0.3627             2   
6      rate_spread           -2.9176           0.0433             2   
7           ln_vix           -1.3371           0.6120             8   

   ADF stat (diff)  p-value (diff)  lags (diff) order  
0          -5.4126          0.0000            0  I(1)  
1          -4.9965          0.0000            1  I(1)  
2          -5.9963          0.0000            0  I(1)  
3          -3.9505          0.0017           10  I(1)  
4          -5.4606          0.0000            0  I(1)  
5         -10.2548      

# Section 3: Full-sample Testing Using ARDL

* ARDL Model
* Bounds Test
* Long-run coefficients
* ECM Results

## ARDL Model

In [230]:
core_lagged = ['ln_total_takeup','capital_ratio', 'award_rate']
fixed = ['eq_asset_r','cpi','rate_spread','ln_vix']

sel_full = ardl_select_order(
    endog=df['ratio'],
    exog=df[core_lagged],
    fixed=df[fixed],
    maxlag=2,
    maxorder=2,
    trend='c',
    ic='bic'
)

res_full = sel_full.model.fit(cov_type='HAC', cov_kwds={'maxlags':2})

print("Selected ARDL order (full sample):", sel_full.model.ardl_order)
print(res_full.summary())


Selected ARDL order (full sample): (2, 2)
                              ARDL Model Results                              
Dep. Variable:                  ratio   No. Observations:                   44
Model:                     ARDL(2, 2)   Log Likelihood                 155.084
Method:               Conditional MLE   S.D. of innovations              0.006
Date:                Wed, 19 Nov 2025   AIC                           -288.169
Time:                        19:05:59   BIC                           -269.054
Sample:                             2   HQIC                          -281.163
                                   44                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2010      0.038      5.279      0.000       0.123       0.279
ratio.L1          1.0463      0.118      8.900      0.000       0.807       1.28

### ARDL Interpretation

The full-sample ARDL captures meaningful long-run relations, but signs on key variables (equity ratio, VIX, award rate) change magnitude or direction compared to the subsample models, indicating that pooling pre- and post-2021 data obscures regime-specific dynamics.

## Bounds Test

In [231]:
# Full sample bounds test
import numpy as np

def run_bounds_test_matrix(res):
    param_names = list(res.params.index)

    # exclude constant, test all long-run level terms
    test_names = [p for p in param_names if p != "const" and "ratio.L1" not in p]

    k = len(test_names)
    R = np.zeros((k, len(param_names)))
    
    for i, name in enumerate(test_names):
        idx = param_names.index(name)
        R[i, idx] = 1

    q = np.zeros(k)

    cov = res.cov_params()
    beta = res.params.values

    W = float((R @ beta - q).T @ np.linalg.inv(R @ cov @ R.T) @ (R @ beta - q))
    F = W / k

    print("====================================")
    print("FULL-SAMPLE Bounds-Type Wald Test")
    print("====================================")
    print(f"Chi-square statistic: {W:.4f}")
    print(f"F-statistic:         {F:.4f}")
    print(f"Number of restrictions: {k}")

    return F

F_full = run_bounds_test_matrix(res_full)


FULL-SAMPLE Bounds-Type Wald Test
Chi-square statistic: 111.1148
F-statistic:         13.8894
Number of restrictions: 8


### Bounds test results for full sample

F-statistic = 13.89 > all critical values

Cointegration exists in the full sample.

* This means:
* Banks maintained a structural long-run relationship between liquidity and balance-sheet/policy variables across the full 2014–2025 period.
* BUT the coefficients probably changed after the ON-RRP regime shift.

## Long-run coefficients

In [232]:
# Long-run coefficients
import pandas as pd

def compute_long_run_coeffs(res, label="Model"):
    params = res.params
    phi = params.get("ratio.L1")
    lam = 1 - phi

    long_run = {}

    for p in params.index:
        if p == "const" or p == "ratio.L1":
            continue
        base = p.split(".")[0]
        long_run.setdefault(base, 0)
        long_run[base] += params[p]

    theta = {var: val / lam for var, val in long_run.items()}

    df_lr = pd.DataFrame({
        "Sum of Level Coeffs": long_run,
        "Long-Run Coefficient (θ)": theta
    })

    print("\n==============================")
    print(f" Long-Run Coefficients: {label}")
    print("==============================")
    print(f"Speed of adjustment λ = {lam:.6f}")
    print(df_lr.round(6))
    print("\n")

    return df_lr

full_summary = compute_long_run_coeffs(res_full, label="Full-Sample ARDL")



 Long-Run Coefficients: Full-Sample ARDL
Speed of adjustment λ = -0.046268
             Sum of Level Coeffs  Long-Run Coefficient (θ)
ratio                  -0.294641                  6.368131
award_rate             -0.005140                  0.111096
eq_asset_r             -2.063155                 44.591308
cpi                    -0.023265                  0.502837
rate_spread             0.013220                 -0.285725
ln_vix                 -0.007538                  0.162925




### Full-sample long run coefficients results

**The adjustment factor here is:**

λ = −0.0463

A negative λ means:

The long-run equilibrium is valid (cointegration confirmed)

The system corrects with oscillation — typical when ARDL(2,2) includes overshooting

“Although the full-sample bounds test confirms cointegration, the implied long-run coefficients are highly unstable, with magnitudes far exceeding those found in either subsample. This strongly suggests that two different regimes operate before and after 2021, validating the decision to estimate separate pre-2021 and post-2021 models.”

## ECM For Full sample

In [233]:
# ECM for full sample
def build_ecm(df, res, long_run_df, label="ECM"):
    import statsmodels.api as sm
    
    theta = long_run_df["Long-Run Coefficient (θ)"].to_dict()

    ECT = df["ratio"].shift(1)
    const = res.params.get("const", 0)
    ECT -= const

    for var, lr_coef in theta.items():
        if var in df.columns:
            ECT -= lr_coef * df[var].shift(1)

    df["ECT_lag"] = ECT

    diff_df = pd.DataFrame()
    diff_df["d_ratio"] = df["ratio"].diff()

    regressors = list(set([p.split(".")[0] for p in res.params.index if p != "const" and p != "ratio.L1"]))

    for var in regressors:
        if var in df.columns:
            diff_df[f"d_{var}"] = df[var].diff()

    diff_df["ECT_lag"] = df["ECT_lag"]
    diff_df = diff_df.dropna()

    y = diff_df["d_ratio"]
    X = sm.add_constant(diff_df.drop(columns=["d_ratio"]))

    ecm_res = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':2})

    print("\n======================================")
    print(f"{label} — Error Correction Model Results")
    print("======================================")
    print(ecm_res.summary())

    return ecm_res

ecm_full = build_ecm(df, res_full, full_summary, label="Full-Sample ECM")



Full-Sample ECM — Error Correction Model Results
                            OLS Regression Results                            
Dep. Variable:                d_ratio   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     8.827
Date:                Wed, 19 Nov 2025   Prob (F-statistic):           6.13e-06
Time:                        19:06:03   Log-Likelihood:                 154.45
No. Observations:                  43   AIC:                            -294.9
Df Residuals:                      36   BIC:                            -282.6
Df Model:                           6                                         
Covariance Type:                  HAC                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

### Full sample ECM results

The full-sample ECM results reveal that, although the ARDL bounds test confirms a long-run relationship among the variables over the full 2014–2025 period, the short-run adjustment dynamics are unstable when the two regimes are pooled together. The error-correction term is positive and significant (ECT = +0.035), indicating that deviations from the long-run equilibrium actually widen rather than converge when the pre-2021 and post-2021 data are combined. This wrong-signed adjustment coefficient suggests that the structural behavior of bank liquidity fundamentally changed after 2021, making the two periods incompatible within a single equilibrium correction mechanism. While several short-run coefficients remain significant—including equity ratio, award rate, rate spread, and CPI—the mixed signs and magnitudes reflect contradictory dynamics across the two regimes. Overall, the full-sample ECM highlights a clear structural break around 2021, reinforcing the necessity of estimating separate pre- and post-2021 models to capture the distinct liquidity adjustment processes before and after the ON-RRP expansion.

## ARDL Model Pre-2021

In [234]:
# ARDL model pre 2021
core_lagged = ['ln_total_takeup','capital_ratio', 'award_rate']  # 2–4 vars max
fixed = ['eq_asset_r','cpi','rate_spread','ln_vix']            # contemporaneous controls

sel = ardl_select_order(
    endog=df_pre['ratio'],
    exog=df_pre[core_lagged],
    fixed=df_pre[fixed],
    maxlag=2,         # quarterly data: 1–2 lags is plenty
    maxorder=2,
    trend='c',
    ic='bic'
)
res_pre = sel.model.fit(cov_type='HAC', cov_kwds={'maxlags':2})
print(sel.model.ardl_order)
print(res.summary())


(1, 0, 2)
                              ARDL Model Results                              
Dep. Variable:                  ratio   No. Observations:                   28
Model:                  ARDL(1, 0, 2)   Log Likelihood                 115.830
Method:               Conditional MLE   S.D. of innovations              0.003
Date:                Wed, 19 Nov 2025   AIC                           -209.660
Time:                        19:06:06   BIC                           -195.405
Sample:                             2   HQIC                          -205.421
                                   28                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0101      0.144      0.070      0.945      -0.294       0.314
ratio.L1               0.8228      0.035     23.662      0.000       0.749       0.896
ln_total_t

### Bounds test for Pre-2021 ARDL

Step 1 — Run ARDL Bounds Test

You need:

- F-statistic
- Compare against Pesaran critical values
- Conclude whether cointegration exists

If cointegration exists → ECM is justified.
If not → you stick to ARDL + differenced short-run model.

In [235]:
import numpy as np

def run_bounds_test_matrix(res_pre):
    param_names = list(res_pre.params.index)
    
    # coefficients to test (exclude constant)
    test_names = [
        'ratio.L1',
        'ln_total_takeup.L0',
        'capital_ratio.L0',
        'capital_ratio.L1',
    #    'award_rate.L0', 
     #   'award_rate.L1',
        'eq_asset_r',
        'cpi',
        'rate_spread',
        'ln_vix'
    ]
    
    k = len(test_names)  # number of restrictions
    
    # Build R matrix: shape (k x n_params)
    R = np.zeros((k, len(param_names)))
    
    for i, name in enumerate(test_names):
        idx = param_names.index(name)
        R[i, idx] = 1

    q = np.zeros(k)  # restriction vector: all = 0

    # Wald test manually
    cov = res_pre.cov_params()
    beta = res_pre.params.values
    
    W = float((R @ beta - q).T @ np.linalg.inv(R @ cov @ R.T) @ (R @ beta - q))  # chi-square version
    F = W / k  # convert to F-statistic for bounds test
    
    print("====================================")
    print("Manual Bounds-Type Wald Test Result")
    print("====================================")
    print(f"Chi-square statistic: {W:.4f}")
    print(f"F-statistic:        {F:.4f}")
    print(f"Number of restrictions: {k}")
    
    return F


In [236]:
F_stat = run_bounds_test_matrix(res_pre)

Manual Bounds-Type Wald Test Result
Chi-square statistic: 7315.2929
F-statistic:        914.4116
Number of restrictions: 8


### ARDL Model Post-2021

In [237]:
# ARDL model post 2021
core_lagged = ['ln_total_takeup','capital_ratio', 'award_rate']  # 2–4 vars max
fixed = ['eq_asset_r','cpi','rate_spread','ln_vix']            # contemporaneous controls

sel = ardl_select_order(
    endog=df_post['ratio'],
    exog=df_post[core_lagged],
    fixed=df_post[fixed],
    maxlag=1,         # quarterly data: 1–2 lags is plenty
    maxorder=1,
    trend='c',
    ic='bic'
)
res_post = sel.model.fit(cov_type='HAC', cov_kwds={'maxlags':2})
print(sel.model.ardl_order)
print(res_post.summary())

(1, 1, 1)
                              ARDL Model Results                              
Dep. Variable:                  ratio   No. Observations:                   16
Model:                  ARDL(1, 1, 1)   Log Likelihood                  79.684
Method:               Conditional MLE   S.D. of innovations              0.001
Date:                Wed, 19 Nov 2025   AIC                           -137.367
Time:                        19:06:10   BIC                           -129.579
Sample:                             1   HQIC                          -137.450
                                   16                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.3696      0.118      3.129      0.026       0.066       0.673
ratio.L1             0.2839      0.047      6.000      0.002       0.162       0.406
capital_ratio.L0  

C:\Users\khang\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\khang\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [238]:
print(res_post.params.index)

Index(['const', 'ratio.L1', 'capital_ratio.L0', 'capital_ratio.L1',
       'award_rate.L0', 'award_rate.L1', 'eq_asset_r', 'cpi', 'rate_spread',
       'ln_vix'],
      dtype='object')


In [239]:
import numpy as np

def run_bounds_test_matrix_post2021(res_post):
    param_names = list(res_post.params.index)
    
    # coefficients to test (exclude constant)
    test_names = [
        'ratio.L1',
        'capital_ratio.L0',
        'capital_ratio.L1',
        'award_rate.L0',
        'award_rate.L1',
        'eq_asset_r',
        'cpi',
        'rate_spread',
        'ln_vix'
    ]
    
    k = len(test_names)  # number of restrictions
    
    # Build R matrix: shape (k x n_params)
    R = np.zeros((k, len(param_names)))
    
    for i, name in enumerate(test_names):
        idx = param_names.index(name)
        R[i, idx] = 1

    q = np.zeros(k)  # restriction vector: all = 0

    # Wald test manually
    cov = res_post.cov_params()
    beta = res_post.params.values
    
    W = float((R @ beta - q).T @ np.linalg.inv(R @ cov @ R.T) @ (R @ beta - q))  # chi-square
    F = W / k  # convert to F-statistic for bounds test
    
    print("====================================")
    print("POST-2021 Bounds-Type Wald Test")
    print("====================================")
    print(f"Chi-square statistic: {W:.4f}")
    print(f"F-statistic:         {F:.4f}")
    print(f"Number of restrictions: {k}")
    
    return F

# Run it
F_post2021 = run_bounds_test_matrix_post2021(res_post)


POST-2021 Bounds-Type Wald Test
Chi-square statistic: 21782.1612
F-statistic:         2420.2401
Number of restrictions: 9


### Notes before comparing long-run coefficients

Post-2021, ARDL model selection drops ON-RRP take-up, suggesting that ON-RRP usage is no longer a determinant of bank liquidity. This aligns with institutional reality: post-2021 ON-RRP usage is dominated by money market funds rather than commercial banks, reducing its direct connection to banks’ balance sheet liquidity management.

## Long run coefficients for pre and post

In [240]:
import pandas as pd

def compute_long_run_coeffs(res, model_name="Model"):
    """
    Computes long-run coefficients θ = sum(level coeffs) / (1 - φ)
    and prints a nice summary.
    """
    params = res.params

    # extract φ (coefficient on ratio.L1)
    phi = params.get("ratio.L1")
    lam = 1 - phi  # speed of adjustment
    
    long_run = {}

    # Identify level coefficients (x.L0, x.L1, x.L2, etc.)
    for p in params.index:
        if p.startswith("ratio"): 
            continue  # skip dependent variable lag

        if ".L" in p or (".L" not in p and p != "const"):
            # extract base variable name, e.g., capital_ratio from capital_ratio.L1
            var_name = p.split(".")[0]
            long_run.setdefault(var_name, 0)
            long_run[var_name] += params[p]

    # compute θ_x = beta_sum / (1 - phi)
    theta = {var: val / lam for var, val in long_run.items()}

    # pretty formatting
    df = pd.DataFrame({
        "Long-Run Coefficient (θ)": theta,
        "Sum of Level Coeffs": long_run
    })

    print(f"\n==============================")
    print(f" Long-Run Coefficients: {model_name}")
    print(f"==============================")
    print(f"Speed of adjustment λ = 1 - φ = {lam:.6f}\n")
    print(df.round(6))
    print("\n")

    return df


# ---- RUN FOR BOTH MODELS ----
pre_summary = compute_long_run_coeffs(res_pre, model_name="Pre-2021 ARDL")
post_summary = compute_long_run_coeffs(res_post, model_name="Post-2021 ARDL")



 Long-Run Coefficients: Pre-2021 ARDL
Speed of adjustment λ = 1 - φ = 0.177151

                 Long-Run Coefficient (θ)  Sum of Level Coeffs
ln_total_takeup                  0.015538             0.002753
capital_ratio                    0.099527             0.017631
eq_asset_r                     -19.712855            -3.492153
cpi                             -0.101734            -0.018022
rate_spread                      0.056541             0.010016
ln_vix                           0.020366             0.003608



 Long-Run Coefficients: Post-2021 ARDL
Speed of adjustment λ = 1 - φ = 0.716100

               Long-Run Coefficient (θ)  Sum of Level Coeffs
capital_ratio                 -0.016750            -0.011995
award_rate                    -0.008938            -0.006401
eq_asset_r                     2.990756             2.141680
cpi                           -0.002627            -0.001881
rate_spread                    0.031225             0.022360
ln_vix                      

In [241]:
def build_ecm(df, res, long_run_df, label="ECM"):
    """
    df: original dataframe (pre- or post-2021)
    res: ARDL fitted model (res_pre or res_post)
    long_run_df: DataFrame with long-run coefficients computed earlier
    label: string for printing
    
    Returns ECM results object.
    """

    # Extract long-run coefficients dictionary
    theta = long_run_df["Long-Run Coefficient (θ)"].to_dict()

    # Build Error Correction Term (ECT)
    ECT = df["ratio"].shift(1)  # start with ratio_{t-1}
    
    # subtract constant (if exists)
    const = res.params.get("const", 0)
    ECT = ECT - const

    # subtract sum(theta_x * X_{t-1})
    for var, lr_coef in theta.items():
        if var in df.columns:
            ECT = ECT - lr_coef * df[var].shift(1)

    df["ECT_lag"] = ECT

    # Build Δ variables
    diff_df = pd.DataFrame()
    diff_df["d_ratio"] = df["ratio"].diff()

    # Add ΔX for all regressors that appear in ARDL
    regressors = [
        col.split(".")[0] for col in res.params.index 
        if col not in ["const", "ratio.L1"]
    ]
    regressors = list(set(regressors))

    for var in regressors:
        if var in df.columns:
            diff_df[f"d_{var}"] = df[var].diff()

    diff_df["ECT_lag"] = df["ECT_lag"]

    # Drop missing rows
    diff_df = diff_df.dropna()

    # Build ECM regression
    y = diff_df["d_ratio"]
    X = diff_df.drop(columns=["d_ratio"])
    X = sm.add_constant(X)

    ecm_res = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':2})

    print("\n======================================")
    print(f"{label} — Error Correction Model Results")
    print("======================================")
    print(ecm_res.summary())

    return ecm_res

In [242]:
ecm_pre = build_ecm(df_pre, res_pre, pre_summary, label="Pre-2021 ECM")


Pre-2021 ECM — Error Correction Model Results
                            OLS Regression Results                            
Dep. Variable:                d_ratio   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     275.7
Date:                Wed, 19 Nov 2025   Prob (F-statistic):           9.76e-18
Time:                        19:06:18   Log-Likelihood:                 111.70
No. Observations:                  27   AIC:                            -207.4
Df Residuals:                      19   BIC:                            -197.0
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

In [243]:
ecm_post = build_ecm(df_post, res_post, post_summary, label="Post-2021 ECM")


Post-2021 ECM — Error Correction Model Results
                            OLS Regression Results                            
Dep. Variable:                d_ratio   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     813.7
Date:                Wed, 19 Nov 2025   Prob (F-statistic):           1.20e-09
Time:                        19:06:18   Log-Likelihood:                 82.860
No. Observations:                  15   AIC:                            -149.7
Df Residuals:                       7   BIC:                            -144.1
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

C:\Users\khang\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)


## Pre and Post 2021 ECM Interpretations

Pre-2021, the error-correction term is negative and highly significant (−0.10), implying that about 10 percent of any deviation from the long-run liquidity equilibrium is corrected each quarter. Short-run dynamics are driven mainly by changes in the equity-to-asset ratio and inflation, while changes in ON-RRP take-up do not have a statistically significant immediate effect on liquidity. This suggests that ON-RRP usage influences bank liquidity primarily through the long-run balance sheet rather than through high-frequency adjustments.

Post-2021, the speed of adjustment increases sharply: the coefficient on the error-correction term (−0.70) indicates that roughly 70 percent of any disequilibrium is eliminated within one quarter. In this regime, short-run changes in the award rate, equity ratio, capital ratio, market volatility (VIX), and the policy spread all have large and statistically significant effects on liquidity, while ON-RRP take-up drops out of the model entirely. This pattern is consistent with the institutional shift after mid-2021, when ON-RRP usage became dominated by money market funds rather than banks. Bank liquidity management appears to respond much more directly to policy rates and risk conditions in this later period.